In [11]:
## Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import scipy.stats as ss

In [12]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # makes figs nicer!

In [13]:
df = pd.read_csv("https://raw.githubusercontent.com/umbertomig/POLI30Dpublic/main/data/BES.csv")

In [14]:
df = pd.read_csv("data/models/classification/bes.csv")

In [21]:
df = pd.read_csv("https://raw.githubusercontent.com/umbertomig/POLI30Dpublic/main/data/pfizer.csv")
df.head(3)

,drug,covid
0,vaccine,1
1,vaccine,1
2,vaccine,1


In [22]:
mod = smf.logit(data = df, formula = "covid ~ drug").fit()

Optimization terminated successfully.
         Current function value: 0.027286
         Iterations 12


In [23]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  covid   No. Observations:                36524
Model:                          Logit   Df Residuals:                    36522
Method:                           MLE   Df Model:                            1
Date:                Fri, 10 Feb 2023   Pseudo R-squ.:                 0.07936
Time:                        11:49:15   Log-Likelihood:                -996.59
converged:                       True   LL-Null:                       -1082.5
Covariance Type:            nonrobust   LLR p-value:                 2.988e-39
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.7161      0.079    -59.759      0.000      -4.871      -4.561
drug[T.vaccine]    -3.0166      0.362     -8.326      0.000      -3.727      -2.306
===================================================================================
"""

Datasets:

- BES (Brexit)
- Pfizer (vaccine)